# Notebook : Ingestion quotidienne de données éoliennes

## 📋 Objectif
Charger de manière incrémentale les données de production éolienne depuis GitHub vers le Lakehouse Bronze.

## 🔄 Logique
1. Identifier la date la plus récente dans Bronze
2. Calculer le jour suivant (date + 1)
3. Télécharger le fichier CSV correspondant depuis GitHub
4. Ajouter les nouvelles données en mode append

## ⚙️ Exécution
- **Fréquence recommandée** : Quotidienne (via pipeline)
- **Durée moyenne** : 50-60 secondes
- **Dépendances** : Repository GitHub public accessible

## 📦 Dépendances
- Lakehouse : LH_Wind_Power_Bronze
- Source : https://github.com/gsoulat/data-training-fabric/tree/main/eolienne

In [19]:
import requests
import pandas as pd
from datetime import datetime, timedelta

StatementMeta(, bdb62b0d-71b4-4c1b-8d02-ceaaa962ad0c, 21, Finished, Available, Finished)

In [20]:
# URL de base du repository GitHub
base_url = "https://raw.githubusercontent.com/gsoulat/data-training-fabric/refs/heads/main/eolienne/"

# Chemin vers la table Bronze
# bronze_table_path = "abfss://8f015743-6f02-47e8-8086-1af3ad394858@onelake.dfs.fabric.microsoft.com/LH_Wind_Power_Bronze.Lakehouse/Tables/dbo/wind_power"
bronze_table_path = "abfss://8f015743-6f02-47e8-8086-1af3ad394858@onelake.dfs.fabric.microsoft.com/724e2441-3064-4829-9858-9434190c2da8/Tables/dbo/wind_power"

StatementMeta(, bdb62b0d-71b4-4c1b-8d02-ceaaa962ad0c, 22, Finished, Available, Finished)

In [21]:
# ⚠️ CELLULE D'INITIALISATION - À exécuter UNE SEULE FOIS
# Cette cellule crée la table Bronze avec le premier fichier de données


initial_date = "20240615"  # Premier jour disponible
initial_url = f"{base_url}{initial_date}_wind_power_data.csv"

print(f"📥 Téléchargement du fichier initial : {initial_url}")

# Télécharger et charger le premier fichier
df_initial = pd.read_csv(initial_url)
df_initial['date'] = pd.to_datetime(df_initial['date'])

print(f"✅ Fichier chargé : {len(df_initial)} lignes")
print(f"📊 Aperçu des données :")
print(df_initial.head())

# Convertir en Spark DataFrame et sauvegarder
df_spark_initial = spark.createDataFrame(df_initial)
df_spark_initial.write.format('delta').mode("overwrite").save(bronze_table_path)

print("✅ Table Bronze initialisée avec succès !")

StatementMeta(, bdb62b0d-71b4-4c1b-8d02-ceaaa962ad0c, 23, Finished, Available, Finished)

📥 Téléchargement du fichier initial : https://raw.githubusercontent.com/gsoulat/data-training-fabric/refs/heads/main/eolienne/20240615_wind_power_data.csv
✅ Fichier chargé : 432 lignes
📊 Aperçu des données :
   production_id       date      time turbine_name  capacity location_name  \
0           6049 2024-06-15  00-00-00    Turbine A      2200    Location 1   
1           6050 2024-06-15  00-00-00    Turbine B      2000    Location 2   
2           6051 2024-06-15  00-00-00    Turbine C      2500    Location 3   
3           6052 2024-06-15  00-10-00    Turbine A      2200    Location 1   
4           6053 2024-06-15  00-10-00    Turbine B      2000    Location 2   

   latitude  longitude    region      status responsible_department  \
0   34.0522  -118.2437  Region A      Online             Operations   
1   36.7783  -119.4179  Region B  Inspection            Maintenance   
2   40.7128   -74.0060  Region C      Online             Operations   
3   34.0522  -118.2437  Region A      O

In [22]:
# Vérifier que la table existe et afficher quelques statistiques
df_check = spark.read.format("delta").load(bronze_table_path)

print(f"📊 Nombre total de lignes : {df_check.count()}")
print(f"📅 Colonnes disponibles : {df_check.columns}")
print(f"🔍 Aperçu des 5 premières lignes :")
df_check.show(5)

StatementMeta(, bdb62b0d-71b4-4c1b-8d02-ceaaa962ad0c, 24, Finished, Available, Finished)

📊 Nombre total de lignes : 432
📅 Colonnes disponibles : ['production_id', 'date', 'time', 'turbine_name', 'capacity', 'location_name', 'latitude', 'longitude', 'region', 'status', 'responsible_department', 'wind_speed', 'wind_direction', 'energy_produced']
🔍 Aperçu des 5 premières lignes :
+-------------+-------------------+--------+------------+--------+-------------+--------+---------+--------+------+----------------------+----------+--------------+---------------+
|production_id|               date|    time|turbine_name|capacity|location_name|latitude|longitude|  region|status|responsible_department|wind_speed|wind_direction|energy_produced|
+-------------+-------------------+--------+------------+--------+-------------+--------+---------+--------+------+----------------------+----------+--------------+---------------+
|         6157|2024-06-15 00:00:00|06-00-00|   Turbine A|    2200|   Location 1| 34.0522|-118.2437|Region A|Online|            Operations|    8.4012|             S|  

In [23]:
# Charger les données existantes depuis Bronze
df_spark = spark.read.format("delta").load(bronze_table_path)

# Convertir en Pandas pour manipulation plus facile
df_pandas = df_spark.toPandas()

print(f"📊 Données actuelles dans Bronze : {len(df_pandas)} lignes")

StatementMeta(, bdb62b0d-71b4-4c1b-8d02-ceaaa962ad0c, 25, Finished, Available, Finished)

📊 Données actuelles dans Bronze : 432 lignes


In [24]:
# Trouver la date la plus récente dans les données
most_recent_date = pd.to_datetime(df_pandas['date'], format="%Y%m%d").max()

# Calculer le jour suivant
next_date = (most_recent_date + timedelta(days=1)).strftime("%Y%m%d")

print(f"📅 Date la plus récente : {most_recent_date.strftime('%Y-%m-%d')}")
print(f"➡️  Prochaine date à charger : {next_date}")

StatementMeta(, bdb62b0d-71b4-4c1b-8d02-ceaaa962ad0c, 26, Finished, Available, Finished)

📅 Date la plus récente : 2024-06-15
➡️  Prochaine date à charger : 20240616


In [25]:
# Construire l'URL du fichier CSV
file_url = f"{base_url}{next_date}_wind_power_data.csv"
print(f"🌐 URL du fichier : {file_url}")

try:
    # Télécharger le CSV depuis GitHub
    df_pandas_new = pd.read_csv(file_url)
    
    # Convertir la colonne date en datetime
    df_pandas_new['date'] = pd.to_datetime(df_pandas_new['date'])
    
    print(f"✅ Nouvelles données téléchargées : {len(df_pandas_new)} lignes")
    print(f"📊 Aperçu des nouvelles données :")
    print(df_pandas_new.head())
    
except Exception as e:
    print(f"❌ Erreur lors du téléchargement : {e}")
    print(f"💡 Cela peut signifier que le fichier pour la date {next_date} n'existe pas encore.")

StatementMeta(, bdb62b0d-71b4-4c1b-8d02-ceaaa962ad0c, 27, Finished, Available, Finished)

🌐 URL du fichier : https://raw.githubusercontent.com/gsoulat/data-training-fabric/refs/heads/main/eolienne/20240616_wind_power_data.csv
✅ Nouvelles données téléchargées : 432 lignes
📊 Aperçu des nouvelles données :
   production_id       date      time turbine_name  capacity location_name  \
0           6481 2024-06-16  00-00-00    Turbine A      2200    Location 1   
1           6482 2024-06-16  00-00-00    Turbine B      2000    Location 2   
2           6483 2024-06-16  00-00-00    Turbine C      2500    Location 3   
3           6484 2024-06-16  00-10-00    Turbine A      2200    Location 1   
4           6485 2024-06-16  00-10-00    Turbine B      2000    Location 2   

   latitude  longitude    region  status responsible_department  wind_speed  \
0   34.0522  -118.2437  Region A  Online             Operations    17.49365   
1   36.7783  -119.4179  Region B  Online             Operations    13.09261   
2   40.7128   -74.0060  Region C  Online             Operations     8.26528   


In [26]:
# Convertir le DataFrame Pandas en Spark DataFrame
df_spark_new = spark.createDataFrame(df_pandas_new, schema=df_spark.schema)

# Ajouter les nouvelles données à la table Bronze (mode append)
df_spark_new.write.format('delta').mode("append").save(bronze_table_path)

print("✅ Données ajoutées avec succès dans le Lakehouse Bronze")
print(f"📊 Total après ajout : {spark.read.format('delta').load(bronze_table_path).count()} lignes")

StatementMeta(, bdb62b0d-71b4-4c1b-8d02-ceaaa962ad0c, 28, Finished, Available, Finished)

✅ Données ajoutées avec succès dans le Lakehouse Bronze
📊 Total après ajout : 864 lignes


In [27]:
df_pandas_wind

StatementMeta(, bdb62b0d-71b4-4c1b-8d02-ceaaa962ad0c, 29, Finished, Available, Finished)

DataFrame[production_id: bigint, date: timestamp, time: string, turbine_name: string, capacity: bigint, location_name: string, latitude: double, longitude: double, region: string, status: string, responsible_department: string, wind_speed: double, wind_direction: string, energy_produced: double]